In [45]:
import numpy as np
import pandas as pd
import sympy as sp
from scipy.linalg import lu

In [46]:
# Define variables
x, y, z = sp.symbols('x y z')

# Define function
f = x**2 + 6*x*y + y**2 - 3*y*z + 4*z**2 - 10*x - 5*y - 21*z

# Gradient (vector of first derivatives)
grad_f = [sp.diff(f, var) for var in (x, y, z)]
print("Gradient:", grad_f)

# Hessian (matrix of second derivatives)
Hessian_f = sp.hessian(f, (x, y, z))
print("Hessian matrix:")
sp.pprint(Hessian_f)

Gradient: [2*x + 6*y - 10, 6*x + 2*y - 3*z - 5, -3*y + 8*z - 21]
Hessian matrix:
⎡2  6   0 ⎤
⎢         ⎥
⎢6  2   -3⎥
⎢         ⎥
⎣0  -3  8 ⎦


This code uses the sympy library for symbolic mathematics in Python.

Define variables:
x, y, z = sp.symbols('x y z')
Creates symbolic variables x, y, and z.

Define function:
f = x**2 + 6*x*y + y**2 - 3*y*z + 4*z**2 - 10*x - 5*y - 21*z
Defines a multivariate quadratic function in terms of x, y, and z.

Gradient:
grad_f = [sp.diff(f, var) for var in (x, y, z)]
Computes the gradient (first derivatives with respect to each variable).

Hessian:
Hessian_f = sp.hessian(f, (x, y, z))
Computes the Hessian matrix (second derivatives with respect to each variable).

Print results:
The gradient and Hessian matrix are printed, showing the rate of change and curvature of the function, respectively.

In [47]:
# Convert to equations = 0
eqs = [sp.Eq(g, 0) for g in grad_f]

# Convert to matrix form A*x = b
A, b = sp.linear_eq_to_matrix(eqs, (x, y, z))

print("Coefficient matrix A =")
sp.pprint(A)

print("Right-hand side vector b =")
sp.pprint(b)


Coefficient matrix A =
⎡2  6   0 ⎤
⎢         ⎥
⎢6  2   -3⎥
⎢         ⎥
⎣0  -3  8 ⎦
Right-hand side vector b =
⎡10⎤
⎢  ⎥
⎢5 ⎥
⎢  ⎥
⎣21⎦


This code takes the gradient of a multivariate function and sets each partial derivative to zero, which is the condition for finding stationary points (such as minima, maxima, or saddle points).

eqs = [sp.Eq(g, 0) for g in grad_f]
Converts each gradient component into an equation set to zero.

A, b = sp.linear_eq_to_matrix(eqs, (x, y, z))
Transforms the system of equations into matrix form: A*x = b, where A is the coefficient matrix and b is the right-hand side vector.

sp.pprint(A) and sp.pprint(b)
Nicely print the coefficient matrix and the right-hand side vector.

This approach is useful for solving systems of linear equations symbolically, especially when finding the critical points of quadratic functions.

In [48]:
# Matrix
A = np.array(A, dtype=float)

# Perform LU decomposition
P,L,U = lu(A)

print("P (permutation matrix):\n", P)
print("L (lower triangular):\n", L)
print("U (upper triangular):\n", U)

# Verify A = P @ L @ U
print("Reconstructed A:\n", P @ L @ U)

# Inverses
L_inv = np.linalg.inv(L)
U_inv = np.linalg.inv(U)

print("L inverse ="); sp.pprint(L_inv)
print("U inverse ="); sp.pprint(U_inv)

# Compute solution x = U^{-1} * L^{-1} * b
x = U_inv @ L_inv @ (P @ b)
print("Solution x =", x)

P (permutation matrix):
 [[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
L (lower triangular):
 [[ 1.          0.          0.        ]
 [ 0.33333333  1.          0.        ]
 [ 0.         -0.5625      1.        ]]
U (upper triangular):
 [[ 6.          2.         -3.        ]
 [ 0.          5.33333333  1.        ]
 [ 0.          0.          8.5625    ]]
Reconstructed A:
 [[ 2.  6.  0.]
 [ 6.  2. -3.]
 [ 0. -3.  8.]]
L inverse =
[[ 1.          0.          0.        ] 
 [-0.33333333  1.          0.        ] 
 [-0.1875      0.5625      1.        ]]
U inverse =
[[ 0.16666667 -0.0625      0.06569343] 
 [ 0.          0.1875     -0.02189781] 
 [ 0.          0.          0.11678832]]
Solution x = Matrix([[2.00000000000000], [1.00000000000000], [3.00000000000000]])


This code demonstrates how to solve a system of linear equations using LU decomposition:

A = np.array(A, dtype=float)
Converts the symbolic coefficient matrix A to a NumPy array of floats for numerical computation.

P, L, U = lu(A)
Performs LU decomposition, breaking A into a permutation matrix P, a lower triangular matrix L, and an upper triangular matrix U.

The code prints each matrix and verifies that multiplying them (P @ L @ U) reconstructs the original matrix A.

L_inv and U_inv are the inverses of L and U, respectively.

The solution to the system A*x = b is computed as x = U_inv @ L_inv @ (P @ b), which uses the inverses and permutation to solve for x.

This approach is a standard numerical method for solving linear systems, especially when symbolic solutions are not practical.

In [49]:
# Sylvester’s criterion: compute leading principal minors
def sylvesters_criterion(H):
    n = Hessian_f.shape[0]
    minors = [Hessian_f[:k,:k].det() for k in range(1, n+1)]
    return minors

minors = sylvesters_criterion(Hessian_f)

print("\nLeading principal minors:")
for i, val in enumerate(minors, start=1):
    print(f"M{i} =", val)

# Classification
if all(m > 0 for m in minors):
    print("\nHessian is Positive Definite → Local Minimum")
elif all(((-1)**(i+1))*m > 0 for i,m in enumerate(minors, start=1)):
    print("\nHessian is Negative Definite → Local Maximum")
elif any(m == 0 for m in minors):
    print("\nTest Inconclusive (some minors = 0)")
else:
    print("\nHessian is Indefinite → Saddle Point")


Leading principal minors:
M1 = 2
M2 = -32
M3 = -274

Hessian is Indefinite → Saddle Point


This code applies Sylvester’s criterion to classify the critical point of a multivariate function using its Hessian matrix:

sylvesters_criterion(H) computes the leading principal minors (determinants of the top-left k×k submatrices) of the Hessian.
It prints each minor.
If all minors are positive, the
Hessian is positive definite (local minimum).
If minors alternate in sign, the Hessian is negative definite (local maximum).
If any minor is zero, the test is inconclusive.
Otherwise, the Hessian is indefinite (saddle point).
This is a standard method for determining the nature of stationary points in multivariate calculus.